In [0]:

%tensorflow_version 1.x
import numpy as np
import keras
from keras.datasets import mnist
import sys
import scipy
#%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K


from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 10))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

def run_model (train_data_indices):
  """Trains and Keras model with the training points specified by train_data_indices and evaluates model on test data
  #Arguments:
      train_data_indices: indices of current training points within X_train_All
  #Returns:
      Test accuracy
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train = keras.utils.to_categorical(y_train, num_classes=10)
  train_size = y_train.shape[0]
  Weight_Decay = 0.01/train_size
  dropout_prob = 0.25
  nb_filters = 40
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 10
  model = Sequential()
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=2, data_format = "channels_first"))
  model.add(Activation('relu'))
	
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))

  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input,K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  print('y test predictions shape: ' + str(y_test_predictions.shape))
  return np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0])


In [0]:
#Active learning parameters/settings
#Experiment starts with balanced training set of 30 points

dropout_prob = 0.25
num_experiments = 3
num_acquisitions = 1000 
num_masks = 50
batch_size = 10

#Keras Model Parameters
num_classes = 10
nb_filters = 40
nb_pool = 2
nb_conv = 4
img_rows = img_cols = 28

In [0]:
#Loading data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
		  delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
		 delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.arange(60000)
test_ind = np.arange(10000)

y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=10)
X_test = np.expand_dims(X_test[test_ind], axis=1)
folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Scripts_v1/"
out_folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Results_v1/"

print('Running')	  
all_tr_ind = []
all_acc = []

#Iterating across experiments, each of which begins with a different training set (that is balanced across classes)
for e in range(2,3):
  acq_file = "RandomAccBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  ind_file = "RandomIndBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  #exp_acc = []
  exp_acc = list(np.load(out_folder_path+acq_file))
  #Initial training/accuracy
  #train_data_indices = list(np.load(folder_path + 'trainindices' + str(e+1) + '.npy'))
  train_data_indices = list(np.load(out_folder_path+ind_file))
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  #Evaluating initial model
  model_results = run_model(train_data_indices)
  print('trained model')
  num_acquisitions = num_acquisitions - batch_size * (len(exp_acc)-1)
  #exp_acc.append(model_results)
  all_acc.append(exp_acc)
  all_tr_ind.append(train_data_indices)
  print('Initial Acc: ' + str(all_acc))

  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    #Random acquisition of new points
    new_train_indices = np.random.choice(pool_indices, batch_size, replace=False)
    train_data_indices = list(train_data_indices)+list(new_train_indices)
    pool_indices = list(set(pool_indices) - set(new_train_indices))    
    #Evaluating model and saving results  
    model_results = run_model(train_data_indices)
    exp_acc.append(model_results)
    all_acc[-1] = exp_acc
    all_tr_ind[-1] = train_data_indices
    print('len train ind: ' + str(len(train_data_indices)))
    print('len pool ind: ' + str(len(pool_indices)))
    np.save(out_folder_path+acq_file, np.array(exp_acc))
    np.save(out_folder_path+ind_file, np.array(train_data_indices))
    print('All Acc: ' + str(all_acc))
    #print('All Ind: ' + str(all_tr_ind))
  print('All Acc: ' + str(all_acc))





Running


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


y test predictions shape: (10000,)
trained model
Initial Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394, 0.8232, 0.8495, 0.8375, 0.8404, 0.8479, 0.8502, 0.8474, 0.8647, 0.8714, 0.8452, 0.8661, 0.8905, 0.8779, 0.8777, 0.8875, 0.9013, 0.9001, 0.8872, 0.8909, 0.9056, 0.8846, 0.8921, 0.9175, 0.894, 0.9086, 0.9137, 0.8975, 0.905, 0.9199, 0.9158, 0.9102, 0.9027, 0.9127, 0.9115, 0.9242, 0.9199, 0.917, 0.9148, 0.9073, 0.9163, 0.9225, 0.9295, 0.9288, 0.9198, 0.9136, 0.9192, 0.9183, 0.936, 0.9089, 0.9259, 0.9258]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


y test predictions shape: (10000,)
len train ind: 680
len pool ind: 59320
All Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394, 0.8232, 0.8495, 0.8375, 0.8404, 0.8479, 0.8502, 0.8474, 0.8647, 0.8714, 0.8452, 0.8661, 0.8905, 0.8779, 0.8777, 0.8875, 0.9013, 0.9001, 0.8872, 0.8909, 0.9056, 0.8846, 0.8921, 0.9175, 0.894, 0.9086, 0.9137, 0.8975, 0.905, 0.9199, 0.9158, 0.9102, 0.9027, 0.9127, 0.9115, 0.9242, 0.9199, 0.917, 0.9148, 0.9073, 0.9163, 0.9225, 0.9295, 0.9288, 0.9198, 0.9136, 0.9192, 0.9183, 0.936, 0.9089, 0.9259, 0.9258, 0.93]]


...

y test predictions shape: (10000,)
len train ind: 1010
len pool ind: 58990
All Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394, 0.8232, 0.8495, 0.8375, 0.8404, 0.8479, 0.8502, 0.8474, 0.8647, 0.8714, 0.8452, 0.8661, 0.8905, 0.8779, 0.8777, 0.8875, 0.9013, 0.9001, 0.8872, 0.8909, 0.9056, 0.8846, 0.8921, 0.9175, 0.894, 0.9086, 0.9137, 0.8975, 0.905, 0.9199, 0.9158, 0.9102, 0.9027, 0.9127, 0.9115, 0.9242, 0.9199, 0.917, 0.9148, 0.9073, 0.9163, 0.9225, 0.9295, 0.9288, 0.9198, 0.9136, 0.9192, 0.9183, 0.936, 0.9089, 0.9259, 0.9258, 0.93, 0.9428, 0.9306, 0.9199, 0.9298, 0.9178, 0.93, 0.9347, 0.9332, 0.9302, 0.936, 0.9244, 0.9262, 0.926, 0.9468, 0.9336, 0.9464, 0.9339, 0.9301, 0.9387, 0.9295, 0.9461, 0.9297, 0.9494, 0.9334, 0.9439, 0.9438, 0.9464, 0.9321, 0.9251, 0.939, 0.9337, 0.9449, 0.9504]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


y test predictions shape: (10000,)
len train ind: 1020
len pool ind: 58980
All Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394, 0.8232, 0.8495, 0.8375, 0.8404, 0.8479, 0.8502, 0.8474, 0.8647, 0.8714, 0.8452, 0.8661, 0.8905, 0.8779, 0.8777, 0.8875, 0.9013, 0.9001, 0.8872, 0.8909, 0.9056, 0.8846, 0.8921, 0.9175, 0.894, 0.9086, 0.9137, 0.8975, 0.905, 0.9199, 0.9158, 0.9102, 0.9027, 0.9127, 0.9115, 0.9242, 0.9199, 0.917, 0.9148, 0.9073, 0.9163, 0.9225, 0.9295, 0.9288, 0.9198, 0.9136, 0.9192, 0.9183, 0.936, 0.9089, 0.9259, 0.9258, 0.93, 0.9428, 0.9306, 0.9199, 0.9298, 0.9178, 0.93, 0.9347, 0.9332, 0.9302, 0.936, 0.9244, 0.9262, 0.926, 0.9468, 0.9336, 0.9464, 0.9339, 0.9301, 0.9387, 0.9295, 0.9461, 0.9297, 0.9494, 0.9334, 0.9439, 0.9438, 0.9464, 0.9321, 0.9251, 0.939, 0.9337, 0.9449, 0.9504, 0.945]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


y test predictions shape: (10000,)
len train ind: 1030
len pool ind: 58970
All Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394, 0.8232, 0.8495, 0.8375, 0.8404, 0.8479, 0.8502, 0.8474, 0.8647, 0.8714, 0.8452, 0.8661, 0.8905, 0.8779, 0.8777, 0.8875, 0.9013, 0.9001, 0.8872, 0.8909, 0.9056, 0.8846, 0.8921, 0.9175, 0.894, 0.9086, 0.9137, 0.8975, 0.905, 0.9199, 0.9158, 0.9102, 0.9027, 0.9127, 0.9115, 0.9242, 0.9199, 0.917, 0.9148, 0.9073, 0.9163, 0.9225, 0.9295, 0.9288, 0.9198, 0.9136, 0.9192, 0.9183, 0.936, 0.9089, 0.9259, 0.9258, 0.93, 0.9428, 0.9306, 0.9199, 0.9298, 0.9178, 0.93, 0.9347, 0.9332, 0.9302, 0.936, 0.9244, 0.9262, 0.926, 0.9468, 0.9336, 0.9464, 0.9339, 0.9301, 0.9387, 0.9295, 0.9461, 0.9297, 0.9494, 0.9334, 0.9439, 0.9438, 0.9464, 0.9321, 0.9251, 0.939, 0.9337, 0.9449, 0.9504, 0.945, 0.9427]]
All Acc: [[0.5315, 0.6145, 0.6442, 0.6626, 0.7484, 0.7696, 0.7115, 0.7471, 0.7832, 0.8036, 0.813, 0.7891, 0.821, 0.8394